<a href="https://colab.research.google.com/github/gd03champ/sqllama/blob/main/TinyLlama_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install accelerate peft bitsandbytes transformers trl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00


In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

In [4]:
# load the required packages.

import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os

In [21]:
load_dataset('burkelibbey/colors')

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['color', 'description'],
        num_rows: 33887
    })
})

In [34]:
# Checking dataset
dataset = load_dataset('wikisql')
dataset

z=1
for x in dataset['train']:
  user_query = "user : " + x['question'] + "\n"          #input (question)
  columns = "columns : " + str(x['table']['header']) + "\n"   #input (table column name)
  columns_type = "columns type : "+ str( x['table']['types'] ) + "\n"     #input (table column type)
  response = x['sql']['human_readable']     #output

  print("<|im_start|>user\n" +  x['question'] + "\n" + "columns : " + str(x['table']['header']) + "\n" + "columns type : "+ str( x['table']['types'] ) + "\n" + "<|im_end|>\n<|im_start|>assistant\n" +  x['sql']['human_readable'] + "\n" + "<|im_end|>\n")

  z-=1
  if z==0:
    break

<|im_start|>user
Tell me what the notes are for South Australia 
columns : ['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes']
columns type : ['text', 'text', 'text', 'text', 'text', 'text']
<|im_end|>
<|im_start|>assistant
SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA
<|im_end|>



In [35]:
dataset="wikisql"
model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
output_model="tinyllama-sql-gd-v1"

### Data preparation

In [36]:
# we need to reformat the data in teh ChatML format.

def formatted_train(input,response)->str:
    return f"<|im_start|>user\n{input}<|im_end|>\n<|im_start|>assistant\n{response}<|im_end|>\n"

In [37]:
def prepare_train_data(data_id):
    data = load_dataset(data_id, split="train")
    data_df = data.to_pandas()
    data_df["text"] = data_df[["question", "table", "sql"]].apply(lambda x: "<|im_start|>user\n" + x['question'] + "\n" + "columns : " + str(x['table']['header']) + "\n" + "columns type : "+ str( x['table']['types'] ) + "\n" + "<|im_end|>\n<|im_start|>assistant\n" +  x['sql']['human_readable'] + "\n" + "<|im_end|>\n", axis=1)
    data = Dataset.from_pandas(data_df)
    return data

In [38]:
data = prepare_train_data(dataset)

In [39]:
data

Dataset({
    features: ['phase', 'question', 'table', 'sql', 'text'],
    num_rows: 56355
})

In [40]:
data[0]

{'phase': 1,
 'question': 'Tell me what the notes are for South Australia ',
 'table': {'caption': '',
  'header': ['State/territory',
   'Text/background colour',
   'Format',
   'Current slogan',
   'Current series',
   'Notes'],
  'id': '1-1000181-1',
  'name': 'table_1000181_1',
  'page_id': '',
  'page_title': '',
  'rows': [['Australian Capital Territory',
    'blue/white',
    'Yaa·nna',
    'ACT · CELEBRATION OF A CENTURY 2013',
    'YIL·00A',
    'Slogan screenprinted on plate'],
   ['New South Wales',
    'black/yellow',
    'aa·nn·aa',
    'NEW SOUTH WALES',
    'BX·99·HI',
    'No slogan on current series'],
   ['New South Wales',
    'black/white',
    'aaa·nna',
    'NSW',
    'CPX·12A',
    'Optional white slimline series'],
   ['Northern Territory',
    'ochre/white',
    'Ca·nn·aa',
    'NT · OUTBACK AUSTRALIA',
    'CB·06·ZZ',
    'New series began in June 2011'],
   ['Queensland',
    'maroon/white',
    'nnn·aaa',
    'QUEENSLAND · SUNSHINE STATE',
    '999·TLG',
  

### Model the Model (not the base version)

In [41]:
def get_model_and_tokenizer(mode_id):

    tokenizer = AutoTokenizer.from_pretrained(mode_id)
    tokenizer.pad_token = tokenizer.eos_token
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto"
    )
    model.config.use_cache=False
    model.config.pretraining_tp=1
    return model, tokenizer

In [49]:
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install accelerate

Looking in indexes: https://pypi.org/simple/


In [50]:
model, tokenizer = get_model_and_tokenizer(model_id)

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

### Setting up the LoRA

In [ ]:
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )

In [ ]:
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=3,
        max_steps=250,
        fp16=True,
        # push_to_hub=True
    )

In [ ]:
trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=1024
    )

Map:   0%|          | 0/33887 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss
10,2.717200
20,2.335900
30,2.026000
40,1.789000
50,1.644000
60,1.556500
70,1.491900
80,1.419800
90,1.397800
100,1.365700


TrainOutput(global_step=250, training_loss=1.5073588218688965, metrics={'train_runtime': 607.2986, 'train_samples_per_second': 26.346, 'train_steps_per_second': 0.412, 'total_flos': 8033688265162752.0, 'train_loss': 1.5073588218688965, 'epoch': 0.47})

### Merging the LoRA with the base model

In [ ]:
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM
import torch
import os

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, load_in_8bit=False,
                                             device_map="auto",
                                             trust_remote_code=True)

model_path = "/content/tinyllama-colorist-v1/checkpoint-250"

peft_model = PeftModel.from_pretrained(model, model_path, from_transformers=True, device_map="auto")

model = peft_model.merge_and_unload()

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32003, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

### Inference from the LLM

In [ ]:
from transformers import GenerationConfig
from time import perf_counter

def generate_response(user_input):

  prompt = formatted_prompt(user_input)

  inputs = tokenizer([prompt], return_tensors="pt")
  generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,
      top_k=5,temperature=0.5,repetition_penalty=1.2,
      max_new_tokens=12,pad_token_id=tokenizer.eos_token_id
  )
  start_time = perf_counter()

  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

  outputs = model.generate(**inputs, generation_config=generation_config)
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  output_time = perf_counter() - start_time
  print(f"Time taken for inference: {round(output_time,2)} seconds")

In [ ]:
def formatted_prompt(question)-> str:
    return f"<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant:"

In [ ]:
def print_color_space(hex_color):
    def hex_to_rgb(hex_color):
        hex_color = hex_color.lstrip('#')
        return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    r, g, b = hex_to_rgb(hex_color)
    print(f'{hex_color}: \033[48;2;{r};{g};{b}m           \033[0m')

In [ ]:
generate_response(user_input='Light Orange color')

user
Light Orange color 
 assistant: #f0901a 
 light
Time taken for inference: 0.35 seconds


In [ ]:
print_color_space('#f0901a ')


#f0901a :            
